In [7]:
import os
from groq import Groq
from dotenv import load_dotenv
from itertools import chain

from pydantic_ai import Agent
from pydantic_ai.result import RunResult

import nest_asyncio
nest_asyncio.apply()



In [11]:
# Load environment variables and initialize the language model
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LOGFIRE_IGNORE_NO_CONFIG'] = '1'

client = Groq(api_key=os.getenv('GROQ_API_KEY'))  # Initialize the Groq client
model_name = "mixtral-8x7b-32768"

def ask_groq(user_input: str):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "You are a helpful AI assistant."},
                  {"role": "user", "content": user_input}]
    )
    return response.choices[0].message.content




In [15]:
store: dict[str, list[dict]] = {}

def create_session_if_not_exists(session_id: str) -> None:
    """Ensures that `session_id` exists in the chat storage."""
    if session_id not in store:
        store[session_id] = []

def get_chat_history(session_id: str) -> list[dict]:
    """Returns the existing chat history."""
    create_session_if_not_exists(session_id)
    return store[session_id]

def store_messages_in_history(session_id: str, messages: list[dict]) -> None:
    """Stores new messages into the chat history."""
    create_session_if_not_exists(session_id)
    store[session_id].extend(messages)

In [16]:

def ask_with_history(user_message: str, user_session_id: str) -> str:
    """Asks the chatbot a question while maintaining chat history."""
    chat_history = get_chat_history(user_session_id)

    # Convert stored messages to Groq API format
    groq_messages = [{"role": "system", "content": "You are a helpful AI assistant."}] + chat_history
    groq_messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
        model=model_name,
        messages=groq_messages
    )

    ai_response = response.choices[0].message.content  # Extract response text
    store_messages_in_history(user_session_id, [{"role": "user", "content": user_message},
                                                {"role": "assistant", "content": ai_response}])

    return ai_response

In [17]:
session_id = 'user_123'

response1 = ask_with_history('Hello! How are you?', session_id)
print('AI:', response1)

AI: Hello! I'm an artificial intelligence, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?


In [18]:
response2 = ask_with_history('What was my previous message?', session_id)
print('AI:', response2)

AI: Your previous message was: "You are a helpful AI assistant. Hello! How are you?" I am unable to retrieve messages that were sent before our current conversation began, but I am ready to help you with any questions or tasks you have now.


In [19]:
# Print the conversation history
print('\nConversation History:')
for message in get_chat_history(session_id):
    print(f'{message["role"]}: {message["content"]}')


Conversation History:
user: Hello! How are you?
assistant: Hello! I'm an artificial intelligence, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?
user: What was my previous message?
assistant: Your previous message was: "You are a helpful AI assistant. Hello! How are you?" I am unable to retrieve messages that were sent before our current conversation began, but I am ready to help you with any questions or tasks you have now.
